In [1]:
from __future__ import division
from __future__ import print_function
import time
import tensorflow as tf
import numpy as np


In [2]:
#set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

import sys, os
import numpy as np
import time
import csv
from sklearn.metrics import confusion_matrix
import scipy.io as sio
import random
import ABIDE_Parser as Reader
import keras

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
import pickle as pkl
import time
import copy
import scipy.spatial.distance
from tqdm import tqdm
from tensorflow.python.ops import array_ops



/home/celery/.pyenv/versions/GAT/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/home/celery/.pyenv/versions/GAT/lib/python3.5/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/celery/.pyenv/versions/GAT/lib/python3.5/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sk

Glorot initializer

In [3]:
def glorot(shape, name=None):
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    var = tf.Variable(initial, name=name)
    return var

In [12]:
# Define shape (e.g. 128 inputs, 64 outputs)
shape = (5, 5)

# Create the variable
weight_var = glorot(shape, name="test_weights")

# Start a session to evaluate the variable
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    weights = sess.run(weight_var)
    print("Weight shape:", weights.shape)
    print("Weight sample:\n", weights)

Weight shape: (5, 5)
Weight sample:
 [[-0.71648014 -0.04834729  0.3869903  -0.33620343 -0.5158652 ]
 [ 0.2996589  -0.43660522  0.7125673  -0.39135537  0.0455662 ]
 [ 0.32520568 -0.21057141  0.5897924  -0.06281775  0.76459396]
 [ 0.02825612  0.46784544 -0.13363522  0.5326885  -0.609742  ]
 [-0.6846892  -0.6376404  -0.4884361   0.31057727 -0.25517952]]


zeros

In [13]:
def zeros(shape, name=None):
    """All zeros."""
    initial = tf.zeros(shape, dtype=tf.float32)
    return tf.Variable(initial, name=name)

In [20]:
shape = (5,5)
a = zeros(shape, name = 'testing')
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(a))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


these are easy

In [21]:
def dot(x, y, sparse=False):
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x,y)
    else:
        res = tf.matmul(x,y)
    return res

def accuracy(preds, labels):
    correct_prediction = tf.equal(tf.round(preds), labels)
    accuracy = tf.cast(correct_prediction, tf.float32)
    return tf.reduce_mean(accuracy)

def tens(shape, name=None):
    initial = tf.constant(10, tf.float32, shape)
    return tf.Variable(initial, name=name)


Gat_layer

In [ ]:
class gat_layer(object):
    def __init__(self, input_dim,F_, placeholders,attn_heads=1,attn_heads_reduction='concat',
                 activation=tf.nn.relu, use_bias=True,name_=''):
        self.dropout_rate = placeholders['dropout']
        self.in_drop = placeholders['in_drop']
        self.name = 'gat_layer'+name_
        self.vars = {}
        self.act = activation
        self.attn_heads = attn_heads  # Number of attention heads (K in the paper)
        self.attn_heads_reduction = attn_heads_reduction  #
        self.bias = use_bias
        self.A = placeholders["adj"]
        self.input_dim = input_dim

        with tf.variable_scope(self.name+'_vars'):
            for i in range(self.attn_heads):
                self.vars['weights_'+str(i)] = glorot([input_dim, F_], name='weights_' + str(i))
                self.vars["attn_self_weights_"+str(i)] = glorot([F_, 1], name='attn_self_weights_' + str(i))
                self.vars["attn_neighs_weights_"+str(i)] = glorot([F_, 1], name='attn_neighs_weights_' + str(i))
        if self.bias:
            self.vars['bias'] = zeros([F_],name='bias')

    def __call__(self, inputs):
        X = inputs
        if self.in_drop != 0.0:
            X = tf.nn.dropout(X, 1-self.in_drop)
        outputs = []
        dense_mask = []

        for head in range(self.attn_heads):
            # Compute inputs to attention network
            kernel = self.vars['weights_'+str(head)]
            features = tf.tensordot(X, kernel, axes=1)  # (N x F')

            # Compute feature combinations
            attention_self_kernel = self.vars["attn_self_weights_"+str(head)]
            attention_neighs_kernel = self.vars["attn_neighs_weights_"+str(head)]
            attn_for_self = tf.tensordot(features, attention_self_kernel, axes=1)
            attn_for_neighs = tf.tensordot(features, attention_neighs_kernel, axes=1)

            # Attention head a(Wh_i, Wh_j) = a^T [[Wh_i], [Wh_j]]
            dense = attn_for_self + tf.transpose(attn_for_neighs, [0,2,1])  # (N x N) via broadcasting

            print("plus:", dense.shape)

            # Add nonlinearty
            dense = tf.nn.leaky_relu(dense,alpha=0.2)

            zero_vec = -9e15*tf.ones_like(dense)
            dense = tf.where(self.A > 0.0, dense, zero_vec)
            dense_mask.append(dense)

            # Apply softmax to get attention coefficients
            dense = tf.nn.softmax(dense)  # (N x N)

            # Apply dropout to features and attention coefficients
            dropout_attn = tf.nn.dropout(dense, 1-self.dropout_rate) # (N x N)
            dropout_feat = tf.nn.dropout(features, 1-self.dropout_rate)  # (N x F')

            # Linear combination with neighbors' features
            node_features = tf.matmul(dropout_attn, dropout_feat)  # (N x F')

            if self.bias:
                node_features += self.vars["bias"]

            # Add output of attention head to final output
            if self.attn_heads_reduction == 'concat':
                outputs.append(self.act(node_features))
            else:
                outputs.append(node_features)

        # Aggregate the heads' output according to the reduction method
        if self.attn_heads_reduction == 'concat':
            output = tf.concat(outputs, axis=-1)  # (N x KF')
        else:
            output = tf.add_n(outputs) / self.attn_heads  # N x F')
            output = self.act(output)

        return output, dense_mask
